In [1]:
import os
import csv

# Path①のディレクトリパス
path1 = f"./data_beta/aspects_and_vectors/岡山"

# latlng.csvのパス
# ここでは、latlng.csvがPath①と同じ階層にあると仮定します。
# 実際のパスに合わせて変更してください。
latlng_csv_path = f"./data_beta/latlng/岡山_latlng_review_exist3.csv"

# 出力するlatlng2.csvのパス
latlng2_csv_path = f"./data_beta/latlng/岡山_latlng_review_top150.csv"

# Step 1: Path①に存在するファイル名からスポット名を抽出
spot_names = set()
for filename in os.listdir(path1):
    if filename.endswith("aspect_from_gpt_cluster_with_embeddings.csv"):
        # ファイル名から「aspect_from_gpt_cluster_with_embeddings.csv」を除去してスポット名を取得
        spot_name = filename.replace("aspect_from_gpt_cluster_with_embeddings.csv", "").strip()
        spot_names.add(spot_name)

print(f"抽出されたスポット名の数: {len(spot_names)}")

# Step 2: latlng.csvからスポット名に対応する行を抽出
matched_rows = []
with open(latlng_csv_path, 'r', encoding='utf-8') as latlng_file:
    reader = csv.reader(latlng_file)
    for row in reader:
        if len(row) < 3:
            continue  # 不完全な行をスキップ
        spot = row[0].strip()
        if spot in spot_names:
            matched_rows.append(row)

print(f"matched_rows に含まれる行の数: {len(matched_rows)}")

# Step 3: matched_rowsをlatlng2.csvに書き込む
with open(latlng2_csv_path, 'w', encoding='utf-8', newline='') as latlng2_file:
    writer = csv.writer(latlng2_file)
    writer.writerows(matched_rows)

print(f"latlng2.csvが正常に作成されました: {latlng2_csv_path}")


抽出されたスポット名の数: 150
matched_rows に含まれる行の数: 150
latlng2.csvが正常に作成されました: ./data_beta/latlng/岡山_latlng_review_top150.csv


In [12]:
# spots_info = {spotname:{lat:lat,lng:lng,aspects:{apsect1:vector1,aspect2:vector,..},spots_aspectsVector:vector,spot_numOfRev:number,spot_url:url}}

test_dict = {"岡山":{"岡山後楽園":{"lat":0.21,"lng":0.54,"aspects":{"景色":[1,3,4],"散策":[3,1,3]},"spots_aspectsVector":[2,1,3],"spot_numOfRev":123,"spot_url":"http://"},"アリオ":{}}}

test_dict2 = {}

test_dict2["岡山"]={}
test_dict2["岡山"]["lat"]=1
test_dict2

{'岡山': {'lat': 1}}

In [19]:
pref = "岡山"
spots_info = {}
latlng_info_path = f"data_beta/latlng/{pref}_latlng_review_top150.csv"
with open(latlng_info_path, 'r', encoding='utf-8') as f_latlng:
    reader = csv.reader(f_latlng)
    for row in reader:
        sn = row[0]
        lat = float(row[1])
        lng = float(row[2])
        spots_info[sn]={}
        spots_info[sn]["lat"] = lat
        spots_info[sn]["lng"] = lng
        spots_info[sn]["apsects"] = {}


In [44]:
def get_pref_spot_info(pref):
    # spots_info = [[spot_name_1, [lat_1,lng_1], [aspects_1],[asp_vectors_1],[spots_aspectsVector_float_1],spot_numOfRev,spot_url], ... ]
    # spots_info = {spotname:{lat:lat,lng:lng,aspects:{apsect1:vector1,aspect2:vector,..},spots_aspectsVector:vector,spot_numOfRev:number,spot_url:url}}
    spots_info = {}
    latlng_info_path = f"data_beta/latlng/{pref}_latlng_review_top150.csv"
    with open(latlng_info_path, 'r', encoding='utf-8') as f_latlng:
        reader = csv.reader(f_latlng)
        for row in reader:
            spot_name = row[0]
            lat = float(row[1])
            lng = float(row[2])
            spots_info[spot_name]={}
            spots_info[spot_name]["lat"] = lat
            spots_info[spot_name]["lng"] = lng
            spots_info[spot_name]["aspects"] = {}
            
    aspect_folder_path = f"./data_beta/aspects_and_vectors/{pref}/"
    read_aspectsVector_path = f"./data_beta/spots_aspect_vector/{pref}aspectVector_fromCluster_GPT.csv"

    with open(read_aspectsVector_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            spot_name = row[0]
            spots_aspectVector_str = row[1]
            if spot_name in spots_info:
                spots_info[spot_name]["spots_aspectsVector"] = [float(value) for value in spots_aspectVector_str.replace("[", "").replace("]", "").replace("\n", "").replace(",","").split()]
            
    #レビュー数を読み込みこむ
    review_num_path  = f"data_beta/number_of_review/{pref}_numOfRview.csv"
    with open(review_num_path,"r",encoding="utf-8") as f_r:
        reader = csv.reader(f_r)
        for row in reader:
            spot_name = row[0]
            numOfrev = row[1]
            if spot_name in spots_info:
                spots_info[spot_name]["numOfRev"] = numOfrev

    #スポットのurlを読み込む
    url_info_path = f"data_beta/url/{pref}_Spots_url_exist_3.csv"
    with open(url_info_path,"r",encoding="utf-8") as f_r:
        reader = csv.reader(f_r)
        for row in reader:
            spot_name = row[0]
            url = row[1]
            if spot_name in spots_info:
                spots_info[spot_name]["spot_url"] = url

    for spot_name,spot_info in spots_info.items():
        aspect_path = aspect_folder_path + spot_name + "aspect_from_gpt_cluster_with_embeddings.csv"
        with open(aspect_path,"r",encoding="utf-8") as f_aspect:
            reader = csv.reader(f_aspect)
            header = next(reader)
            rows = list(reader)
        for row in rows:
            aspect = row[0]
            vector_str = row[5]
            vector_float = [float(value) for value in vector_str.replace("[", "").replace("]", "").replace("\n", "").split(",")]
            spot_info["aspects"][aspect] = vector_float
    print(f"ファイルが見つかりました!! : {pref}")
    # except FileNotFoundError:
    #     print(f"ファイルが見つかりませんでした。:{pref}")
    #     spots_info = []

    return spots_info


returned_ = get_pref_spot_info("岡山")

ファイルが見つかりました!! : 岡山


In [ ]:
aspects_keys_view = list(returned_["岡山後楽園"]["aspects"].keys())
all_aspect_dict = {}
for spot_name,spot_info in returned_.items():
    for aspect in list(spot_info["aspects"].keys()):
        print(aspect)

In [ ]:
list(returned_["岡山後楽園"]["aspects"].keys())